# **New Aviation Fleet Acquisition Risk Analysis**


## *Introduction*



This analysis uses the aviation dataset to allow us to quantify the overall risk associated with different types of aircraft and identify which configurations have historically demonstrated safer performance.Specifically, using aircraft category, engine type, number of engines, and manufacturer (make) relate to historical accident outcomes, including injury severity, aircraft damage, and counts of fatal, serious, minor, and uninjured passengers. By interpreting injury and damage patterns across manufacturers and engine setups, we can provide clear guidance on which aircraft types pose lower operational risk and are therefore better suited for our company’s entry into commercial and private aviation services.

In the cell below we import the relevant libraries to be used for the Analysis.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


This part of code shows how to load the Data, select the necessary relevant columns to be used to achieve our goal for this analysis

In [32]:

df = pd.read_csv(
    "AviationData.csv",
    encoding="latin1",
    low_memory=False
)

# Loading Relevant columns for the study
cols = [
    "Aircraft.Category",
    "Injury.Severity",
    "Aircraft.damage",
    "Number.of.Engines",
    "Engine.Type",
    "Make",
    "Total.Fatal.Injuries",
    "Total.Serious.Injuries",
    "Total.Minor.Injuries",
    "Total.Uninjured",
]

We have already selected the relevant columns we need successfully.Now, we are now looping through each column name in the list cols to compare their existence with the provided dataset anf helps us to identify which column exactly is missing.

In [33]:
for c in cols:
    if c not in df.columns:
        raise ValueError(f"Missing required column: {c}")
df = df[cols].copy()

### *Dealing with Missing Values*
This code fills missing values in specific categorical columns with the label "Unknown". Creates a list of column names
 that are categorical (text-based, not numeric) and loops through each column name in the list and replaces missing values with Unknown and assigns it back to the dataframe. This keeps all rows ensuring no data loss.

In [35]:
cat_cols = ["Aircraft.Category", "Injury.Severity", "Aircraft.damage", "Engine.Type", "Make"]
for c in cat_cols:
    df[c] = df[c].fillna("Unknown")


This function is designed to clean and standardize the “number of engines” field so that it becomes a usable numeric value. Try is a protective block that ensures the code does not crush if it fails.

In [42]:
#Parse Number.of.Engines into integer where possible
def parse_num_engines(x):
    try:
        if pd.isna(x):
            return np.nan
        if isinstance(x, (int, float)):
            if np.isnan(x):
                return np.nan
            return int(x)
        s = str(x).strip().lower()
        if s == '':
            return np.nan
        #simpler mapping to values
        map_words = {"one":1,"single":1,"two":2,"twin":2,"three":3,"four":4}
        for w,v in map_words.items():
            if w in s:
                return v
            #digits
            import re
        m = re.search(r"\d+", s)
        if m:
            return int(m.group())
        return np.nan
    except Exception:
        return np.nan
df["no.of.engines_parsed"] = df["Number.of.Engines"].apply(parse_num_engines)

    #Fill with mode if still missing
if df["no.of.engines_parsed"].dropna().empty:
    mode_eng = 1
else:
    mode_eng = int(df["no.of.engines_parsed"].mode().iloc[0])
df["no.of.engines_parsed"] = df["no.of.engines_parsed"].fillna(mode_eng).astype(int)
df["no.of.engines_imputed"] = df["no.of.engines_parsed"].isna()

# Numeric injury fill and imputation flags
num_inj_cols = ["Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured"]
for c in num_inj_cols:
    df[c + "_imputed_flag"] = df[c].isna()
    df[c] = df[c].fillna(0).astype(int)

# Derived flags/metrics
df["total_injuries_reported"] = (df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"]
                                 + df["Total.Minor.Injuries"] + df["Total.Uninjured"])
df["had_fatal"] = (df["Total.Fatal.Injuries"] > 0).astype(int)
df["had_serious"] = (df["Total.Serious.Injuries"] > 0).astype(int)


### *Aggregating Data*


In [43]:
def aggregate_group(df, by_cols, min_records=30):
    g = df.groupby(by_cols).agg(
        records=('Make','count'),
        total_fatal=('Total.Fatal.Injuries','sum'),
        total_serious=('Total.Serious.Injuries','sum'),
        total_minor=('Total.Minor.Injuries','sum'),
        total_uninjured=('Total.Uninjured','sum'),
        incidents_with_fatal=('had_fatal','sum'),
        incidents_with_serious=('had_serious','sum'),
    ).reset_index()
    g["avg_fatal_per_record"] = g["total_fatal"] / g["records"]
    g["prop_incidents_with_fatal"] = g["incidents_with_fatal"] / g["records"]
    g["avg_serious_per_record"] = g["total_serious"] / g["records"]
    # optionally filter small-sample groups
    g_filtered = g[g["records"] >= min_records].copy()
    return g, g_filtered

make_agg, make_agg_filtered = aggregate_group(df, ["Make"], min_records=30)
category_agg, category_agg_filtered = aggregate_group(df, ["Aircraft.Category"], min_records=10)
engine_type_agg, engine_type_agg_filtered = aggregate_group(df, ["Engine.Type"], min_records=10)
engines_count_agg, engines_count_agg_filtered = aggregate_group(df, ["no.of.engines_parsed"], min_records=10)
